In [326]:
import pandas as pd
import re
from ast import Yield
from itertools import zip_longest
import os
import pytesseract
from pdf2image import convert_from_path, convert_from_bytes
import cv2
import re
import pandas as pd
from datetime import datetime
from pdf2image import convert_from_path,convert_from_bytes
from pdf2image.exceptions import (
PDFInfoNotInstalledError,
PDFPageCountError,
PDFSyntaxError
)


class Gaudi:
    pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
    def __init__(self):
        self.config= '--psm 4  -c preserve_interword_spaces=1 tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789.[]|,,.~â ÃÂç'
        self.tesseract_language = "por"
        self.lista_produtos = []
        self.imagem = 'C:\\Users\\Crawler\\Documents\\DEVPYTHON\\brands\\imagens\\'
        self.path = 'C:\\Users\\Crawler\\Documents\\DEVPYTHON\\brands\\gaudi.pdf'
        self.dataatual = str(datetime.today().strftime('%Y-%m-%d %H:%M'))
        self.idmarca = '14'

    def converter_imgpdf(self):
        images = convert_from_path(self.path, 200, poppler_path=r'C:\\Users\\Crawler\\Documents\\appconversorarquivos\\poppler-21.10.0\\bin')
        for i in range(len(images)):
            images[i].save(self.imagem+'\gaudi'+ str(i) +'.jpg', 'JPEG')


    def reader_imagem(self):
        lista_produtos = []
        imgs = os.listdir(self.imagem)
        for im in imgs: 
            if 'gaudi' in im:
                img = cv2.imread(os.path.join(self.imagem,im))
                #imagemgray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) #Convertendo para rgb
                imagemgray = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #Convertendo para rgb
                texto = pytesseract.image_to_string(imagemgray, lang= self.tesseract_language,config=self.config)
                valores = texto.split()
                saldos_produtos = re.compile(r'([0]?[o]?\d+\.\d+)')
                referencias_produtos = re.compile(r'([8]\d{3}[0]?)')
                for valor in valores:
                    produto_saldo = saldos_produtos.findall(valor)
                   
                    produto_referencia = referencias_produtos.findall(valor)

                    for x, y in zip_longest(produto_referencia, produto_saldo):
                        dict_items = {}

                        if x != None:
                            try:
                                dict_items['SKU'] = x
                            except:
                                dict_items['SKU'] = 'naotem'

                        elif y !=None:

                            try:
                                dict_items['SALDO'] = y
                            except:
                                dict_items['SALDO'] = 0

                        
                        lista_produtos.append(dict_items)
            
        data = pd.DataFrame(lista_produtos)
        data['SKU'] = data['SKU'].fillna(method='pad')
        data['SALDO'] = data['SALDO'].astype(str)
    
        gaudi= data[data['SALDO'] != 'nan']
        gaudi.to_excel("saldogauditeste.xlsx")


     
        
gaudi = Gaudi()
gaudi.reader_imagem()
